<a href="https://colab.research.google.com/github/Subhitsha123/Algorithim/blob/main/Tzu_Chi_data_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
from google.colab import files
uploaded = files.upload()

Saving Member Attendances Report (cth.member.attendance.report) (2).csv to Member Attendances Report (cth.member.attendance.report) (2) (1).csv
Saving SGO Data Tzu Chi SEEN@Bukit Batok (Mar 2025)_21.05.2025 (1).xls to SGO Data Tzu Chi SEEN@Bukit Batok (Mar 2025)_21.05.2025 (1) (1).xls
Saving Members (cth.user).csv to Members (cth.user) (2).csv


In [105]:
import pandas as pd
activities_df = pd.read_csv("Member Attendances Report (cth.member.attendance.report) (2).csv")
members_df = pd.read_csv("Members (cth.user).csv")

In [106]:

# First, filter only 'Physical' activities
physical_activities = activities_df[activities_df['AAP Domain'].str.lower() == 'physical']

# Count number of physical sessions per phone number
physical_count = physical_activities.groupby('Phone').size().reset_index(name='Physical_Activity_Count')

# Convert both to string before merging
members_df['Contact Number'] = members_df['Contact Number'].astype(str)
physical_count['Phone'] = physical_count['Phone'].astype(str)

# Now safely merge
merged_df = pd.merge(members_df, physical_count, left_on='Contact Number', right_on='Phone', how='left')

# Fill missing physical counts with 0
merged_df['Physical_Activity_Count'] = merged_df['Physical_Activity_Count'].fillna(0).astype(int)


In [107]:
# Clean and prepare the 'Phone' column
activities_df['Phone'] = activities_df['Phone'].astype(str).str.strip()
activities_df['Session Detail/Date'] = pd.to_datetime(activities_df['Session Detail/Date'], errors='coerce')

# Group by phone and get last session date
last_session = (
    activities_df
    .groupby('Phone')['Session Detail/Date']
    .max()
    .reset_index()
    .rename(columns={'Session Detail/Date': 'Last_Activity_Date'})
)

# Compute days inactive
from datetime import datetime
last_session['Activity_Days_Inactive'] = (datetime.now() - last_session['Last_Activity_Date']).dt.days
final_df = pd.merge(merged_df, sgo, on='NRIC', how='left', indicator=True)


final_df['Contact Number'] = final_df['Contact Number'].astype(str).str.strip()
final_df = final_df.drop(columns=['Activity_Days_Inactive'], errors='ignore')

final_df = pd.merge(
    final_df,
    last_session[['Phone', 'Activity_Days_Inactive']],
    left_on='Contact Number',
    right_on='Phone',
    how='left'
)

# Clean up
final_df = final_df.drop(columns=['Phone'], errors='ignore')
final_df['Activity_Days_Inactive'] = final_df['Activity_Days_Inactive'].fillna(999).astype(int)


In [108]:
merged_df = merged_df.drop(
    columns=[
        'Is NOK?',
        'Services',
        'Last Measured Weight',
        'Last Measured Blood Pressure',
        'Activity Type'
    ],
    errors='ignore'
)

merged_df.count()

,0
Age,5352
Centre (as client),5348
CFS,1147
Contact Number,5411
Full Name,5348
Is Client,5352
Is interested to volunteer for the centre?,3744
Within Boundary,5340
NRIC,5348
Housing Type,4339


In [110]:
print("Merged columns:")
for col in final_df.columns:
    print(col)

Merged columns:
Age
Centre (as client)
CFS_x
Contact Number
Full Name
Is Client
Is interested to volunteer for the centre?
Is NOK?
Services
Within Boundary
NRIC
Housing Type
Last F2F Date
Last Measured Blood Pressure
Last Participation Date
Last Measured Weight
Marital Status
Social Risk Factor(s)
Activity Type
Phone_x
Physical_Activity_Count
Date of Assessment/Visit
Name of AAC
Name of Senior
UIN
Senior GRC
Senior Division
Year of Birth
Gender
Ethnicity
Language spoken/understood
Number of Risk Factor(s)
Social Support Recommendation
CFS_y
AAP Recommendation
Postal Code
Block
Street Name
Floor
Unit Number
Working Status
Senior Contact Number
Contact Number of Next of Kin
Title of Next of Kin
Name of Next of Kin
Preferred day to call/visit
Preferred time to call/visit
Short term memory - recall 3 unrelated items (e.g. watch, table, apple) after 5 mins.
Cognitive skills for daily decision making
BADL Choose the activities that senior is unable to perform.
IADLChoose the activities that 

In [111]:
merged_df = merged_df[merged_df['Centre (as client)'] == 'Tzu Chi SEEN @ Bukit Batok']
merged_df = merged_df[merged_df['Within Boundary'] == 'Yes']

In [112]:
merged_df = merged_df[merged_df['Full Name'].notna() & (merged_df['Full Name'].str.strip() != '')]


In [113]:
merged_df.count()

,0
Age,1146
Centre (as client),1146
CFS,897
Contact Number,1146
Full Name,1146
Is Client,1146
Is interested to volunteer for the centre?,1143
Within Boundary,1146
NRIC,1146
Housing Type,1044


In [114]:
sgo = pd.read_excel("SGO Data Tzu Chi SEEN@Bukit Batok (Mar 2025)_21.05.2025 (1).xls", engine='xlrd')

In [115]:
final_df = pd.merge(merged_df, sgo, on='NRIC', how='left', indicator = True)
no_sgo_data = final_df[final_df['_merge'] == 'left_only']
no_sgo_data



,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Within Boundary,NRIC,Housing Type,...,"Senior has risk of harm to self or others: [Self-reported] Ask: ""Is there anyone around you that makes you feel afraid or unsafe?""",Indicate if senior is a care recipient or a main caregiver,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,_merge
8,59.0,Tzu Chi SEEN @ Bukit Batok,NaN,97366084,Wang Bing Grace,No,No,Yes,S2608742B,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10,85.0,Tzu Chi SEEN @ Bukit Batok,5,93691873,Chew Ah Tai (Passed On),Yes,No,Yes,S0544316D,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
14,57.0,Tzu Chi SEEN @ Bukit Batok,NaN,87829901,Yeo Chiew Gek,No,No,Yes,S1830925D,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
17,77.0,Tzu Chi SEEN @ Bukit Batok,NaN,85252298,Palanisamy so Kalimuthu,Yes,No,Yes,S2121906A,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18,82.0,Tzu Chi SEEN @ Bukit Batok,NaN,97550913,Hoo Oi Kiau,Yes,No,Yes,S0955324Z,4-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,67.0,Tzu Chi SEEN @ Bukit Batok,5,66986476,Rahim Bin Man,Yes,NaN,Yes,S1280118A,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1133,82.0,Tzu Chi SEEN @ Bukit Batok,NaN,nan,Ramlah Binti Baba,Yes,No,Yes,S2037820D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1134,51.0,Tzu Chi SEEN @ Bukit Batok,NaN,98148915,Poh Geong Sen,No,Yes,Yes,S7455630B,4-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1142,70.0,Tzu Chi SEEN @ Bukit Batok,1-3,93868526,N Rajaletchumiy,Yes,Yes,Yes,S0018436E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [116]:
final_df['_merge'].value_counts()


,count
_merge,
both,870
left_only,276
right_only,0


In [117]:
no_sgo_members = final_df[final_df['_merge'] == 'left_only']
no_sgo_members

,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Within Boundary,NRIC,Housing Type,...,"Senior has risk of harm to self or others: [Self-reported] Ask: ""Is there anyone around you that makes you feel afraid or unsafe?""",Indicate if senior is a care recipient or a main caregiver,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,_merge
8,59.0,Tzu Chi SEEN @ Bukit Batok,NaN,97366084,Wang Bing Grace,No,No,Yes,S2608742B,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
10,85.0,Tzu Chi SEEN @ Bukit Batok,5,93691873,Chew Ah Tai (Passed On),Yes,No,Yes,S0544316D,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
14,57.0,Tzu Chi SEEN @ Bukit Batok,NaN,87829901,Yeo Chiew Gek,No,No,Yes,S1830925D,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
17,77.0,Tzu Chi SEEN @ Bukit Batok,NaN,85252298,Palanisamy so Kalimuthu,Yes,No,Yes,S2121906A,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
18,82.0,Tzu Chi SEEN @ Bukit Batok,NaN,97550913,Hoo Oi Kiau,Yes,No,Yes,S0955324Z,4-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,67.0,Tzu Chi SEEN @ Bukit Batok,5,66986476,Rahim Bin Man,Yes,NaN,Yes,S1280118A,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1133,82.0,Tzu Chi SEEN @ Bukit Batok,NaN,nan,Ramlah Binti Baba,Yes,No,Yes,S2037820D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1134,51.0,Tzu Chi SEEN @ Bukit Batok,NaN,98148915,Poh Geong Sen,No,Yes,Yes,S7455630B,4-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1142,70.0,Tzu Chi SEEN @ Bukit Batok,1-3,93868526,N Rajaletchumiy,Yes,Yes,Yes,S0018436E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [118]:
# from datetime import datetime

# # 1. Convert session date to datetime
# activities_df['Session Detail/Date'] = pd.to_datetime(activities_df['Session Detail/Date'], errors='coerce')

# # 2. Filter only physical activities
# physical_activities = activities_df[activities_df['AAP Domain'].str.lower() == 'physical']

# # 3. Get last session date per NRIC
# last_session = (
#     physical_activities
#     .groupby('NRIC')['Session Detail/Date']
#     .max()
#     .reset_index()
#     .rename(columns={'Session Detail/Date': 'Last_Activity_Date'})
# )

# # 4. Compute inactivity in days
# last_session['Days_Inactive'] = (datetime.now() - last_session['Last_Activity_Date']).dt.days

# last_session.rename(columns={'Days_Inactive': 'Activity_Days_Inactive'}, inplace=True)

# final_df = pd.merge(final_df, last_session[['NRIC', 'Activity_Days_Inactive']], on='NRIC', how='left')

# final_df['Activity_Days_Inactive'] = final_df['Activity_Days_Inactive'].fillna(999).astype(int)


In [119]:
def categorize_risk(row):
    cfs = pd.to_numeric(row.get('CFS_x'), errors='coerce')
    srf_raw = str(row.get('Social Risk Factor(s)', '')).strip()
    if srf_raw == '>1':
        srf = 2
    else:
        srf = pd.to_numeric(srf_raw, errors='coerce')

    housing = str(row.get('Housing Type', '')).lower()
    days_inactive = pd.to_numeric(row.get('Activity_Days_Inactive'), errors='coerce')
    marital_status = str(row.get('Marital Status', '')).lower()
    alone = any(x in marital_status for x in ['single', 'widowed', 'divorced'])

    # New SGO factors
    low_mood = str(row.get('Indication of low mood', '')).strip().lower() == 'yes'
    suicidal_thoughts = str(row.get('Have you thought of harming yourself in the last 3 months?', '')).strip().lower() == 'yes'
    loneliness = pd.to_numeric(row.get('Degree of loneliness'), errors='coerce')
    social_cutoff = str(row.get('Has limited or no social interaction', '')).strip().lower() == 'yes'
    unstable_condition = str(row.get('Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)', '')).strip().lower() == 'yes'

    # Flags
    frail = pd.notna(cfs) and cfs >= 6
    moderate_frail = pd.notna(cfs) and cfs in [4, 5]
    risky_srf = pd.notna(srf) and srf >= 1
    poor_housing = any(x in housing for x in ['1-rm hdb', '2-rm hdb', '3-rm hdb'])
    inactive = pd.notna(days_inactive) and days_inactive > 30

    # R1: urgent cases
    if suicidal_thoughts or (loneliness >= 3 and low_mood and alone and frail):
        return 'R1'

    # R2: concerning
    elif (frail or moderate_frail) and (low_mood or risky_srf or unstable_condition or social_cutoff) and (alone or poor_housing or inactive):
        return 'R2'

    # R3: minor risk
    elif (frail or moderate_frail or low_mood or risky_srf or social_cutoff or poor_housing) and inactive:
        return 'R3'

    # R4: low risk
    else:
        return 'R4'


In [120]:
final_df['Risk Level'] = final_df.apply(categorize_risk, axis=1)


In [121]:
final_df['Risk Level'].value_counts().reindex(['R1', 'R2', 'R3', 'R4']).fillna(0).astype(int)


,count
Risk Level,
R1,0
R2,14
R3,0
R4,1132


In [122]:
df2 = final_df[final_df['Risk Level'] == 'R2']
df2

,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Within Boundary,NRIC,Housing Type,...,Indicate if senior is a care recipient or a main caregiver,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,_merge,Risk Level
243,80.0,Tzu Chi SEEN @ Bukit Batok,4,93952903,Rohani Bin Yusof,Yes,No,Yes,S0655425C,3-RM HDB,...,Care Recipient,No,No,NaN,NaN,Not lonely,No,NaN,both,R2
284,72.0,Tzu Chi SEEN @ Bukit Batok,6,86809836,NG Goon Lam,Yes,No,Yes,S0106595E,3-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2
293,61.0,Tzu Chi SEEN @ Bukit Batok,4,85803461,Segeran S/O Rajoo Krishna,Yes,No,Yes,S1641092F,3-RM HDB,...,Care Recipient,No,No,No,NaN,Frequently (weekly but less than daily),Yes,Senior does not have hobbies,both,R2
377,78.0,Tzu Chi SEEN @ Bukit Batok,4,65695136,Tang Bee Ngo,Yes,No,Yes,F2796356X,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R2
390,76.0,Tzu Chi SEEN @ Bukit Batok,4,94812289,Tan Kee Sin,Yes,Yes,Yes,S0563326E,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R2
515,64.0,Tzu Chi SEEN @ Bukit Batok,4,87122344,Jasman Bin Jalani,Yes,No,Yes,S1500336G,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R2
560,75.0,Tzu Chi SEEN @ Bukit Batok,4,86228000,Tay Eng Hua,Yes,Yes,Yes,S0116900I,2-RM HDB,...,NaN,NaN,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2
617,93.0,Tzu Chi SEEN @ Bukit Batok,4,68961532,Ong Choon Huay,Yes,Yes,Yes,S0868928H,4-RM HDB,...,Care Recipient,No,No,NaN,NaN,Not lonely,No,NaN,both,R2
680,73.0,Tzu Chi SEEN @ Bukit Batok,4,92305028,Moothy S/O Appudurai,Yes,No,Yes,S2000346D,2-RM HDB,...,NaN,NaN,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2
686,94.0,Tzu Chi SEEN @ Bukit Batok,5,87602745,Lily Tilaka,Yes,No,Yes,S2114742G,2-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2


In [133]:
print(final_df.columns.tolist())

['Full Name', 'NRIC', 'CFS_x', 'Social Risk Factor(s)', 'Housing Type', 'Marital Status', 'Indication of low mood', 'Have you thought of harming yourself in the last 3 months?', 'Degree of loneliness', 'Has limited or no social interaction', 'Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)', 'Risk Level']


In [134]:
final_df.columns = final_df.columns.str.strip()  # removes leading/trailing spaces
final_df.columns = final_df.columns.str.replace(r'\s+', ' ', regex=True)  # collapse weird spacing


In [142]:
print(last_session.columns.tolist())
last_session.head()


['Phone', 'Last_Activity_Date', 'Activity_Days_Inactive']


,Phone,Last_Activity_Date,Activity_Days_Inactive
0,5694809.0,2025-04-19,53
1,60122551.0,2025-05-23,19
2,60167085669.0,2025-05-30,12
3,62638770.0,2025-05-31,11
4,62759559.0,2025-04-17,55


In [144]:
print(final_df['Contact_Number'].dropna().astype(str).str.strip().head(10))
print(last_session['Phone'].dropna().astype(str).str.strip().head(10))

KeyError: 'Contact_Number'

In [140]:
for col in final_df.columns:
    if 'Inactive' in col:
        print(f"Found: '{col}'")

    else:
      print("No")

No
No
No
No
No
No
No
No
No
No
No
No


In [129]:
final_cols = [
    'Full Name', 'NRIC',
    'CFS_x', 'Social Risk Factor(s)', 'Housing Type',
    'Marital Status', 'Days_Inactive',
    'Indication of low mood',
    'Have you thought of harming yourself in the last 3 months?',
    'Degree of loneliness',
    'Has limited or no social interaction',
    'Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)',
    'Risk Level',
    'Activity_Days_Inactive'
]


In [130]:
final_df = final_df[[col for col in final_cols if col in final_df.columns]]


In [131]:
final_df

,Full Name,NRIC,CFS_x,Social Risk Factor(s),Housing Type,Marital Status,Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,"Instability of Conditions - conditions that make cognitive, ADL, mood or behaviour patterns unstable (fluctuating, precarious or deteriorating)",Risk Level
0,Tan Gek Eng,S1219664D,NaN,0,4-RM HDB,MARRIED,NaN,NaN,NaN,NaN,NaN,R4
1,Leo Ah Tah,S0821703C,1-3,0,2-RM HDB,PARTNER/SIGNIFICANT OTHER,No,NaN,Not lonely,No,No,R4
2,Ong Sok Hian,S1342713E,5,0,2-RM HDB,WIDOWED,No,NaN,Not lonely,No,No,R4
3,Mohamad Bin Kamis,S1396067D,5,0,2-RM HDB,MARRIED,No,NaN,Not lonely,No,No,R4
4,Soh Lay Har,S1458236C,5,0,3-RM HDB,MARRIED,No,NaN,Not lonely,No,No,R4
...,...,...,...,...,...,...,...,...,...,...,...,...
1141,Soh Boon Loke,S1252142A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R4
1142,N Rajaletchumiy,S0018436E,1-3,NaN,NaN,SINGLE,NaN,NaN,NaN,NaN,NaN,R4
1143,Lim Kim Kee(NUP),S1323675E,1-3,NaN,NaN,MARRIED,No,NaN,Not lonely,No,No,R4
1144,Lee May Kin,S1641967B,1-3,0,NaN,MARRIED,No,NaN,Not lonely,No,No,R4


In [123]:
df2 = final_df[final_df['Risk Level'] == 'R2']
df2

,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Within Boundary,NRIC,Housing Type,...,Indicate if senior is a care recipient or a main caregiver,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,_merge,Risk Level
243,80.0,Tzu Chi SEEN @ Bukit Batok,4,93952903,Rohani Bin Yusof,Yes,No,Yes,S0655425C,3-RM HDB,...,Care Recipient,No,No,NaN,NaN,Not lonely,No,NaN,both,R2
284,72.0,Tzu Chi SEEN @ Bukit Batok,6,86809836,NG Goon Lam,Yes,No,Yes,S0106595E,3-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2
293,61.0,Tzu Chi SEEN @ Bukit Batok,4,85803461,Segeran S/O Rajoo Krishna,Yes,No,Yes,S1641092F,3-RM HDB,...,Care Recipient,No,No,No,NaN,Frequently (weekly but less than daily),Yes,Senior does not have hobbies,both,R2
377,78.0,Tzu Chi SEEN @ Bukit Batok,4,65695136,Tang Bee Ngo,Yes,No,Yes,F2796356X,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R2
390,76.0,Tzu Chi SEEN @ Bukit Batok,4,94812289,Tan Kee Sin,Yes,Yes,Yes,S0563326E,2-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R2
515,64.0,Tzu Chi SEEN @ Bukit Batok,4,87122344,Jasman Bin Jalani,Yes,No,Yes,S1500336G,3-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R2
560,75.0,Tzu Chi SEEN @ Bukit Batok,4,86228000,Tay Eng Hua,Yes,Yes,Yes,S0116900I,2-RM HDB,...,NaN,NaN,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2
617,93.0,Tzu Chi SEEN @ Bukit Batok,4,68961532,Ong Choon Huay,Yes,Yes,Yes,S0868928H,4-RM HDB,...,Care Recipient,No,No,NaN,NaN,Not lonely,No,NaN,both,R2
680,73.0,Tzu Chi SEEN @ Bukit Batok,4,92305028,Moothy S/O Appudurai,Yes,No,Yes,S2000346D,2-RM HDB,...,NaN,NaN,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2
686,94.0,Tzu Chi SEEN @ Bukit Batok,5,87602745,Lily Tilaka,Yes,No,Yes,S2114742G,2-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R2


In [124]:
final_df


,Age,Centre (as client),CFS_x,Contact Number,Full Name,Is Client,Is interested to volunteer for the centre?,Within Boundary,NRIC,Housing Type,...,Indicate if senior is a care recipient or a main caregiver,Family or close friends report feeling overwhelmed by senior's illness.,"Living situation:Senior is a caregiver looking after someone who require high care needs (e.g. elderly who is main caregiver to someone who is ADL dependent, with behavioural issues, etc.)",Indication of low mood,Have you thought of harming yourself in the last 3 months?,Degree of loneliness,Has limited or no social interaction,What programmes would senior be interested in or want to see?,_merge,Risk Level
0,69.0,Tzu Chi SEEN @ Bukit Batok,NaN,97503088,Tan Gek Eng,Yes,No,Yes,S1219664D,4-RM HDB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,R4
1,84.0,Tzu Chi SEEN @ Bukit Batok,1-3,68960112,Leo Ah Tah,Yes,No,Yes,S0821703C,2-RM HDB,...,NaN,NaN,No,No,NaN,Not lonely,No,Senior does not have hobbies,both,R4
2,65.0,Tzu Chi SEEN @ Bukit Batok,5,94663498,Ong Sok Hian,Yes,No,Yes,S1342713E,2-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,"Physical activities (e.g. group exercise, team...",both,R4
3,65.0,Tzu Chi SEEN @ Bukit Batok,5,67767203,Mohamad Bin Kamis,Yes,No,Yes,S1396067D,2-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,"Physical activities (e.g. group exercise, team...",both,R4
4,65.0,Tzu Chi SEEN @ Bukit Batok,5,88846202,Soh Lay Har,Yes,No,Yes,S1458236C,3-RM HDB,...,Care Recipient,No,No,No,NaN,Not lonely,No,"Spiritual activities (e.g. meditation, prayers)",both,R4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141,68.0,Tzu Chi SEEN @ Bukit Batok,NaN,97352221,Soh Boon Loke,Yes,No,Yes,S1252142A,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,R4
1142,70.0,Tzu Chi SEEN @ Bukit Batok,1-3,93868526,N Rajaletchumiy,Yes,Yes,Yes,S0018436E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only,R4
1143,67.0,Tzu Chi SEEN @ Bukit Batok,1-3,91641641,Lim Kim Kee(NUP),Yes,No,Yes,S1323675E,NaN,...,NaN,NaN,No,No,NaN,Not lonely,No,"Physical activities (e.g. group exercise, team...",both,R4
1144,60.0,Tzu Chi SEEN @ Bukit Batok,1-3,98559363,Lee May Kin,Yes,No,Yes,S1641967B,NaN,...,NaN,NaN,No,No,NaN,Not lonely,No,"Spiritual activities (e.g. meditation, prayers)",both,R4


In [125]:
print(final_df.columns.tolist())


['Age', 'Centre (as client)', 'CFS_x', 'Contact Number', 'Full Name', 'Is Client', 'Is interested to volunteer for the centre?', 'Within Boundary', 'NRIC', 'Housing Type', 'Last F2F Date', 'Last Participation Date', 'Marital Status', 'Social Risk Factor(s)', 'Phone', 'Physical_Activity_Count', 'Date of Assessment/Visit', 'Name of AAC', 'Name of Senior', 'UIN', 'Senior GRC', 'Senior Division', 'Year of Birth', 'Gender', 'Ethnicity', 'Language spoken/understood', 'Number of Risk Factor(s)', 'Social Support Recommendation', 'CFS_y', 'AAP Recommendation', 'Postal Code', 'Block', 'Street Name', 'Floor', 'Unit Number', 'Working Status', 'Senior Contact Number', 'Contact Number of Next of Kin', 'Title of Next of Kin', 'Name of Next of Kin', 'Preferred day to call/visit', 'Preferred time to call/visit', 'Short term memory - recall 3 unrelated items (e.g. watch, table, apple) after 5 mins.', 'Cognitive skills for daily decision making', 'BADL Choose the activities that senior is unable to perfo